<a href="https://colab.research.google.com/github/olonok69/NEO4J/blob/main/NEO4J_03_Recommendations_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendations: Part 1

In this notebook you will learn how to make recommendations using Neo4j. 

Execute the code to import the libraries (remember to unset Reset all runtimes before running):

In [ ]:
!pip install py2neo==4.1.3 pandas matplotlib sklearn

     |████████████████████████████████| 71kB 2.9MB/s 
     |████████████████████████████████| 194kB 7.8MB/s 
     |████████████████████████████████| 256kB 19.2MB/s 
     |████████████████████████████████| 133kB 18.6MB/s 
  Created wheel for py2neo: filename=py2neo-4.1.3-py2.py3-none-any.whl size=89805 sha256=60957767ac2977681428baecaa2efcb5b26fd19bb4285a81816f434368f19686
  Stored in directory: /root/.cache/pip/wheels/49/b4/f8/df22db284f7ec2f6c26f87a87ff38c6c8ffa84baf2930ef0fe
  Created wheel for neo4j-driver: filename=neo4j_driver-1.6.3-cp36-cp36m-linux_x86_64.whl size=525046 sha256=dd916ab7fe6f0ee823d53d6c7609f00da0f86c0f0aa47ab494c5af561dfdfaae
  Stored in directory: /root/.cache/pip/wheels/e3/3d/6f/b1afb54585268913bb38334adaa58c2938911edc85102803a1
  Created wheel for neotime: filename=neotime-1.0.0-cp36-none-any.whl size=14714 sha256=705c2c1259d1767a4b3b070836f4d944d030011b58021136a4c729e0e9648b1b
  Stored in directory: /root/.cache/pip/wheels/19/1c/63/e1777cae4998cb32c5ae5bb1901d

In [ ]:
from py2neo import Graph
import pandas as pd

import matplotlib 
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', 100)

Next, create a connection to your Neo4j Sandbox, just as you did previously when you set up your environment. 

<div align="left">
    <img src="https://github.com/neo4j-contrib/training-v2/blob/master/Courses/DataScience/notebooks/images/sandbox-citations.png?raw=1" alt="Citation Sandbox"/>
</div>

Update the cell below to use the IP Address, Bolt Port, and Password, as you did previously.

In [ ]:
# Change the line of code below to use the IP Address, Bolt Port, and Password of your Sandbox.
# graph = Graph("bolt://<IP Address>:<Bolt Port>", auth=("neo4j", "<Password>")) 
 
from py2neo import Graph 
graph = Graph("bolt://100.26.229.35:34820", auth=("neo4j", "dares-problems-concurrence"))

##  Finding popular authors

Since we're going to make collaborator suggestions find authors who have written the most articles so that we have some data to work with.

In [ ]:
popular_authors_query = """
MATCH (author:Author)
RETURN author.name, size((author)<-[:AUTHOR]-()) AS articlesPublished
ORDER BY articlesPublished DESC
LIMIT 10
"""

graph.run(popular_authors_query).to_data_frame()

,author.name,articlesPublished
0,Peter G. Neumann,89
1,Peter J. Denning,80
2,Moshe Y. Vardi,72
3,Pamela Samuelson,71
4,Bart Preneel,65
5,Vinton G. Cerf,56
6,Barry W. Boehm,53
7,Mark Guzdial,49
8,Edwin R. Hancock,47
9,Josef Kittler,46


Pick one of these authors...

In [ ]:
author_name = "Peter G. Neumann"

Retrieve the articles they've published and how many citations they've received:

In [ ]:
author_articles_query = """
MATCH (:Author {name: $authorName})<-[:AUTHOR]-(article)
RETURN article.title AS article, article.year AS year, size((article)<-[:CITED]-()) AS citations
ORDER BY citations DESC
LIMIT 20
"""

graph.run(author_articles_query,  {"authorName": author_name}).to_data_frame()

,article,year,citations
0,"The foresight saga, redux",2012,2
1,Security by obscurity,2003,2
2,Risks of automation: a cautionary total-system perspective of our cyberfuture,2016,1
3,Crypto policy perspectives,1994,1
4,Risks of National Identity Cards,2001,1
5,"Computers, ethics, and values",1991,1
6,Are dependable systems feasible,1993,1
7,Information system security redux,2003,1
8,The foresight saga,2006,1
9,Robust open-source software,1999,1


Find the author's collaborators:

In [ ]:
collaborations_query = """
MATCH (:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor)
RETURN coauthor.name AS coauthor, count(*) AS collaborations
ORDER BY collaborations DESC
LIMIT 10
"""

graph.run(collaborations_query,  {"authorName": author_name}).to_data_frame()

,coauthor,collaborations
0,Lauren Weinstein,3
1,Whitfield Diffie,3
2,Susan Landau,3
3,Steven Michael Bellovin,2
4,Matt Blaze,2
5,Rebecca T. Mercuri,2
6,Alfred Z. Spector,1
7,Seymour E. Goodman,1
8,David Lorge Parnas,1
9,Douglas Miller,1


How would you suggest some future collaborators for this author? One way is by looking at the collaborators of their collaborators!

In [ ]:
collaborations_query = """
MATCH (author:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor),
      (coauthor)<-[:AUTHOR]-()-[:AUTHOR]->(coc)
WHERE not((coc)<-[:AUTHOR]-()-[:AUTHOR]->(author)) AND coc <> author      
RETURN coc.name AS coauthor, count(*) AS collaborations
ORDER BY collaborations DESC
LIMIT 10
"""

graph.run(collaborations_query,  {"authorName": author_name}).to_data_frame()

,coauthor,collaborations
0,John Ioannidis,10
1,Scott Bradner,9
2,Angelos D. Keromytis,8
3,John Kelsey,7
4,Virgil D. Gligor,5
5,David Wagner,4
6,Peter Wolcott,4
7,Ran Canetti,4
8,Gerald Jay Sussman,4
9,David K. Gifford,4


Each of these people have collaborated with someone that Peter has worked with before, so they might be able to do an introduction.

## Exercise

1. Can you find the top 20 suggested collaborators for 'Brian Fitzgerald' instead of 'Peter G. Neumann'?
2. How many of these potential collaborators have collaborated with Brian's collaborators more than 3 times?

Keep the results of this exercise handy as they may be useful for the quiz at the end of this module.

In [ ]:
query = """
MATCH (author:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor),
      (coauthor)<-[:AUTHOR]-()-[:AUTHOR]->(coc)
WHERE not((coc)<-[:AUTHOR]-()-[:AUTHOR]->(author)) AND coc <> author      
RETURN coc.name AS coauthor, count(*) AS collaborations
ORDER BY collaborations DESC
LIMIT 20
"""

params = {"authorName": "Brian Fitzgerald", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

,coauthor,collaborations
0,Holger Giese,5
1,Robert C. Seacord,4
2,Chris Jensen,4
3,Grace A. Lewis,4
4,Gabriel A. Moreno,4
5,Judith A. Stafford,4
6,Kurt C. Wallnau,4
7,Ivar Jacobson,3
8,Klaus Pohl,3
9,Nicolás López,3


In [ ]:
query = """
MATCH (author:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor),
      (coauthor)<-[:AUTHOR]-()-[:AUTHOR]->(coc)
WHERE not((coc)<-[:AUTHOR]-()-[:AUTHOR]->(author)) AND coc <> author      
WITH coc, count(*) AS collaborations
WHERE collaborations > 3
RETURN count(*)
"""
graph.run(query, params).to_data_frame()

,count(*)
0,7
